### step 3 (I) - data preprocessing for RNN model
1. load data
2. sort data
3. deal with na values
4. transform data format for RNN model
5. create .npy for RNN

因資料量過大，Google Colab 只能一次處理一種資料，測試資料或訓練資料擇一。清完 RAM 才能繼續跑下一種資料。

In [1]:
import pandas as pd
from datetime import timedelta, datetime
import re
import warnings
import numpy as np

pd.set_option('display.max_columns', None)  # Set the maximum number of columns to be displayed to None
warnings.filterwarnings("ignore")

### 1. load data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# df = pd.read_csv('/content/drive/MyDrive/BDA/期末報告/g3/data_preprocessing/midpoint_data/g2_after_filter_train.csv')
df = pd.read_csv('/content/drive/MyDrive/BDA/期末報告/g3/data_preprocessing/midpoint_data/g2_after_filter_test.csv')
display(df)

,ShopMemberId,Behavior,Tunnel,Device,PageType,UnitPrice,Qty,TotalSalesAmount,UTMSource,HitTime,EventTime,tag,viewproduct,selectcontent,viewcategory,viewmainpage,add,viewpromotiondetail,viewecoupondetail,checkout,search,viewcustompage,register,purchase,viewcoupondetail,viewarticledetail,UTMSource_Class,UTMMedium,UTMName
0,DRJY1ofby/c7hM+fpZTCzrMNHmEBYcK9NeGKIFPKHS8=,add,App,iOS APP,NaN,750.0,1.0,NaN,(direct),1665243341887,1665243472445,AA,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,NaN,NaN
1,fB/AOARpmRM+5wn8P1q0IWKdZdhQCTxbTXO+8pqNNtg=,add,App,iOS APP,NaN,1050.0,1.0,NaN,(direct),1665846228408,1665848447339,AA,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,NaN,NaN
2,2RnJPOGAenfeIREzppKZkWu4u6kuRV3ulAyKlthcnew=,add,App,Android APP,NaN,3590.0,1.0,NaN,(direct),1666558299612,1666559785833,AA,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,NaN,NaN
3,2RnJPOGAenfeIREzppKZkWu4u6kuRV3ulAyKlthcnew=,add,App,Android APP,NaN,1020.0,1.0,NaN,(direct),1666558299612,1666559809592,AA,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,NaN,NaN
4,areqhtibRC1QnhaiJLQx96O1sg4lUfA31Kn04Xiu3/U=,add,App,iOS APP,NaN,2190.0,1.0,NaN,(direct),1664809613460,1664809885139,AA,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3828287,ebnu2zZm5eSdqDNlpum0+fhP7h7RBKGbelMVKC8m8C0=,viewpromotiondetail,App,iOS APP,NaN,NaN,NaN,NaN,app_sharing,1679751544251,1679751549462,AA,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,cpc,SalePage[-8706710]
3828288,ebnu2zZm5eSdqDNlpum0+fhP7h7RBKGbelMVKC8m8C0=,viewpromotiondetail,App,iOS APP,NaN,NaN,NaN,NaN,app_sharing,1679822963409,1679824451875,AA,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,cpc,SalePage[-8706710]
3828289,yWHYpxTgH9dh6uiBmMHXdAQoOVdkZBHTJfnYHKRiMvo=,viewpromotiondetail,App,iOS APP,NaN,NaN,NaN,NaN,app_sharing,1680258486766,1680259076318,AA,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,cpc,SalePage[-8736131]
3828290,CqEDZRUEdeuf2sG63O9O/H5FdUWoHMI5OLL2yq+K4RU=,viewpromotiondetail,App,iOS APP,NaN,NaN,NaN,NaN,SMS,1679002154683,1679002188498,AL,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,link,0116-0119_300coupon


### 2. sort data

In [7]:
df.sort_values(by=['EventTime'], inplace=True)
df.reset_index(inplace=True)
df

,index,ShopMemberId,Behavior,Tunnel,Device,PageType,UnitPrice,Qty,TotalSalesAmount,UTMSource,HitTime,EventTime,tag,viewproduct,selectcontent,viewcategory,viewmainpage,add,viewpromotiondetail,viewecoupondetail,checkout,search,viewcustompage,register,purchase,viewcoupondetail,viewarticledetail,UTMSource_Class,UTMMedium,UTMName
0,482206,5mOKidjiAh353vheZLMTr17/nfreVkHOTTyrtTnOq7I=,viewmainpage,App,Android APP,NaN,NaN,NaN,NaN,(direct),1664582528495,1664582530055,AP,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN
1,663816,5mOKidjiAh353vheZLMTr17/nfreVkHOTTyrtTnOq7I=,selectcontent,App,Android APP,Home,NaN,NaN,NaN,(direct),1664582528495,1664582532098,AP,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN
2,341549,5mOKidjiAh353vheZLMTr17/nfreVkHOTTyrtTnOq7I=,selectcontent,App,Android APP,Home,NaN,NaN,NaN,(direct),1664582528495,1664582545968,AP,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN
3,628855,5mOKidjiAh353vheZLMTr17/nfreVkHOTTyrtTnOq7I=,viewcategory,App,Android APP,NaN,NaN,NaN,NaN,(direct),1664582528495,1664582547064,AP,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN
4,32145,5mOKidjiAh353vheZLMTr17/nfreVkHOTTyrtTnOq7I=,viewcategory,App,Android APP,NaN,NaN,NaN,NaN,(direct),1664582528495,1664582576333,AP,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3828287,3730564,HXRhFZn6m2BSkrsm+gV+X9IF5MM76k4bla7M5lPnb0Y=,viewcategory,Web,Desktop,NaN,NaN,NaN,NaN,gkw,1680306565986,1680309017128,AA,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,cpc,ecsite
3828288,3718783,HXRhFZn6m2BSkrsm+gV+X9IF5MM76k4bla7M5lPnb0Y=,viewcategory,Web,Desktop,NaN,NaN,NaN,NaN,gkw,1680306565986,1680309048691,AA,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,cpc,ecsite
3828289,3811422,HXRhFZn6m2BSkrsm+gV+X9IF5MM76k4bla7M5lPnb0Y=,selectcontent,Web,Desktop,SalePageCategory,NaN,NaN,NaN,gkw,1680306565986,1680309082551,AA,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,cpc,ecsite
3828290,3295865,HXRhFZn6m2BSkrsm+gV+X9IF5MM76k4bla7M5lPnb0Y=,viewproduct,Web,Desktop,NaN,2380.0,1.0,NaN,gkw,1680306565986,1680309083624,AA,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,cpc,ecsite


### 3. deal with na values
- remove rows out of the expected time horizon
- remove rows that have na values & replace by 0 if necessary

#### (1) remove rows out of the expected time horizon

In [8]:
# check time 
pd.Series([df['EventTime'].max(), df['EventTime'].min()]).apply(lambda x : pd.to_datetime(x, unit='ms') + timedelta(hours=8))

0   2023-04-01 08:31:34.222
1   2022-10-01 08:02:10.055
dtype: datetime64[ns]

In [9]:
# set period ending time
end_time = datetime(2023, 3, 31, 16, 0, 0)
end_time_unix = int(end_time.timestamp()) * 1000
print("end time unix", end_time_unix, type(end_time_unix))

end time unix 1680278400000 <class 'int'>


In [10]:
# find a breakpoint, 在這個 index 之後都是理想期間之外的行為資料
breakPoint = 0 
for index, row in df.iterrows():
    if row['EventTime'] >= end_time_unix:
        print(index, row['EventTime'])
        breakPoint = index
        break

3825135 1680278413313


In [11]:
# check time to make sure that the breakpoint is correct
pd.Series([df[:breakPoint]['EventTime'].max(), df[:breakPoint]['EventTime'].min()]).apply(lambda x : pd.to_datetime(x, unit='ms') + timedelta(hours=8))

0   2023-03-31 23:59:55.127
1   2022-10-01 08:02:10.055
dtype: datetime64[ns]

In [12]:
# remove data not in the period
df = df[:breakPoint]
print("rows left", df.shape[0])

rows left 3825135


In [13]:
# save time correct version of df to avoid repeating the time-consuming process
# df.to_csv('/content/drive/MyDrive/BDA/期末報告/g3/data_preprocessing/midpoint_data/g2_after_filter_train_time_correct.csv', index=False)
df.to_csv('/content/drive/MyDrive/BDA/期末報告/g3/data_preprocessing/midpoint_data/g2_after_filter_test_time_correct.csv', index=False)

#### [ 資料斷點：可以直接從這裡開始，讀取已處理好、已存的資料.csv ]

In [14]:
# df = pd.read_csv('/content/drive/MyDrive/BDA/期末報告/g3/data_preprocessing/midpoint_data/g2_after_filter_train_time_correct.csv')
df = pd.read_csv('/content/drive/MyDrive/BDA/期末報告/g3/data_preprocessing/midpoint_data/g2_after_filter_test_time_correct.csv')

#### (2) remove rows that have na values & replace by 0 if necessary

In [15]:
target_columns = ['ShopMemberId', 'tag', 'Behavior', 'Tunnel', 'Device', 'PageType', 'UnitPrice', 'TotalSalesAmount', 'UTMSource', 'EventTime', 'UTMMedium', 'UTMName']
special_columns = ['PageType', 'UnitPrice', 'TotalSalesAmount']   # columns that have to replace by 0 if there's na value
for col in target_columns:
    if col in special_columns:
        df[col].fillna(0, inplace=True)
    else:
        df.dropna(subset=[col], inplace=True)
    print("rows left", df.shape[0])

rows left 3825135
rows left 3825135
rows left 3825135
rows left 3825135
rows left 3825135
rows left 3825135
rows left 3825135
rows left 3825135
rows left 3822073
rows left 3822073
rows left 1877302
rows left 1814557


In [16]:
# check na and None values to make sure that the previous operation worked correctly
target_columns = ['ShopMemberId', 'tag', 'Behavior', 'Tunnel', 'Device', 'PageType', 'UnitPrice', 'TotalSalesAmount', 'UTMSource', 'EventTime', 'UTMMedium', 'UTMName']
for col in target_columns:
    has_na = df[col].isna().any()
    has_none = df[col].isnull().any()
    print(col, ":")
    print(has_na)
    print(has_none)

ShopMemberId :
False
False
tag :
False
False
Behavior :
False
False
Tunnel :
False
False
Device :
False
False
PageType :
False
False
UnitPrice :
False
False
TotalSalesAmount :
False
False
UTMSource :
False
False
EventTime :
False
False
UTMMedium :
False
False
UTMName :
False
False


### 4. transform data format for RNN model
- frequency encoding
- make y a list
- normalize columns with big range or standard deviation

#### (1) frequency encoding
- check value counts to observe categories
- combine similar strings into one category
- check value counts to make sure that the number of categories has decreased, and it was combined successfully
- frequency encoding using the new categories
- leave only the encoded columns

In [17]:
def check_value_counts(df):
    check_all_values = ['UTMSource', 'UTMName']
    skip = ['index', 'ShopMemberId', 'PageType', 'UnitPrice', 'TotalSalesAmount', 'EventTime', 'Unnamed: 0']
    for column in df:
        if column in skip:
            pass
        elif column in check_all_values:
            print(column)
            pd.set_option("display.max_rows", None)
            display(df[column].value_counts())
            pd.set_option("display.max_rows", 10)
        else:
            print(column)
            print(df[column].value_counts())
        print()

#### check value counts to observe categories

In [18]:
check_value_counts(df)



Behavior
selectcontent          571583
viewproduct            416150
viewmainpage           322815
viewcategory           310954
viewecoupondetail       45445
viewpromotiondetail     42605
add                     42117
viewcustompage          37246
checkout                11932
search                   7175
purchase                 5203
viewarticledetail         877
viewcoupondetail          278
register                  177
Name: Behavior, dtype: int64

Tunnel
App    1663554
Web     151003
Name: Tunnel, dtype: int64

Device
iOS APP        1109735
Android APP     553819
MobileWeb       100953
Desktop          50050
Name: Device, dtype: int64



Qty
1.0    474260
2.0       312
3.0        40
5.0        30
4.0        20
Name: Qty, dtype: int64


UTMSource


brand.com                534684
Audience                 430103
FB_CA                    253290
SMS                      128440
System_Inform            121179
TP                        59491
LINEOA                    56057
insider                   36495
app_sharing               35290
MemberPresentReward       30146
affiliate                 25832
gkw                       23466
google                    22301
omnichat                  20455
PersonalizedMarketing     10541
brandcom                   7422
mweb                       6018
facebook                   4490
LINE                       1607
GOOGLE_CA                  1594
instagram                  1038
(direct)                    740
AD_CA                       649
gdn                         629
CRM_POP                     463
all                         454
line                        407
lineoa                      372
bing                        342
FB                          163
AutoNotification            154
IGS     


HitTime
1677292348639    988
1665718280110    581
1676400687912    574
1677547263030    565
1677513623912    526
                ... 
1666847654139      1
1675861999950      1
1679236229412      1
1672012125716      1
1670369979137      1
Name: HitTime, Length: 187182, dtype: int64


tag
AA    1217821
AP     506479
AL      90257
Name: tag, dtype: int64

viewproduct
0    1398407
1     416150
Name: viewproduct, dtype: int64

selectcontent
0    1242974
1     571583
Name: selectcontent, dtype: int64

viewcategory
0    1503603
1     310954
Name: viewcategory, dtype: int64

viewmainpage
0    1491742
1     322815
Name: viewmainpage, dtype: int64

add
0    1772440
1      42117
Name: add, dtype: int64

viewpromotiondetail
0    1771952
1      42605
Name: viewpromotiondetail, dtype: int64

viewecoupondetail
0    1769112
1      45445
Name: viewecoupondetail, dtype: int64

checkout
0    1802625
1      11932
Name: checkout, dtype: int64

search
0    1807382
1       7175
Name: search, dtype: int64



0112-0129_coupon                                                                          211741
0112-0129_cnygame                                                                         160405
0112-0129_cnysale                                                                          83027
eCoupon_Expiring                                                                           59063
s23-vippreview_m                                                                           53713
1216-1218_viponly                                                                          51088
0823_outletsale                                                                            49724
TradesOrder                                                                                38333
Timberland                                                                                 25813
MemberTier_2                                                                               25111
0116-0119_300coupon           

#### combine similar strings into one category

In [19]:
combine_value_dict = {'UTMSource': ['line', 'google', 'youtube', 'facebook', 'instagram', '.safeframe.googlesyndication.com'], 'UTMName': ['SalePage', 'menu_line']}

# 為了加速運算，數量太多且一定不可能需要替換的字串就不用進行 combine_value_dict 的字串比較與對應
# # for training data
# UTMSource_skip = ['System_Inform', 'AutoNotification', 'Audience', 'TP', 'coupon', 'SMS', 'omnichat', 'app_sharing', 'gkw', 'hiddenlink', 'affiliate']
# UTMName_skip = ['折價券到期(前1天通知)', 'onsale', 'eCoupon_Expiring', '0324_gogr500', 'acquisition', 'Price_Reduction', 
# 'SS22', 'ss2210off', '8/11-8/14_outletseasonoff', 'CouponExpire', '220618', '0308_coupon500', 'summer31', 'ecsite', 
# 'TradesOrder', '0330_coupon', 'summersale', '0419_coupoun', 'Timberland', 'SS22VIP',                                                                                  
# '11hoursonly', 'pre618', '72146', 'ssnew', '618encore', '0823_outletsale', '0328-0331_s22staffonlymn']
# for testing data
UTMSource_skip = ['brand.com', 'Audience', 'SMS', 'System_Inform', 'TP', 'insider', 'app_sharing', 'MemberPresentReward', 
'affiliate', 'gkw', 'omnichat', 'PersonalizedMarketing']
UTMName_skip = ['0112-0129_coupon', '0112-0129_cnygame', '0112-0129_cnysale', 'eCoupon_Expiring', 's23-vippreview_m', '1216-1218_viponly', '0823_outletsale', 'TradesOrder',
'Timberland', 'MemberTier_2', '0116-0119_300coupon', '126390', 'ecsite', 'acquisition', 's23-vippreview_w', 'gift', '226500', '動態化', 'outlet', '128733', '125465', 
'1103-1109_d11_coupon500', '1027-1113_D11appdownload500', '121531', 'Price_Reduction', '120730', 'boots', '221356', '滿額好禮', '127515', '1111KVnew', '119291', '1212_d12sale']

for key, value in combine_value_dict.items():
    for index, row in df.iterrows():
        if key == 'UTMSource':
            if row[key] == 'FB_CA' or 'fb_' in row[key].lower():
                df.at[index, key] = 'facebook'
            elif row[key] == 'LINEOA':
                df.at[index, key] = 'line'
            elif row[key] == 'google':
                pass
            elif row[key] in UTMSource_skip:
                pass
            else:
                for v in value:
                    print(v, row[key])
                    if v.lower() in row[key].lower():
                        df.at[index, key] = v
                        break
        elif key == 'UTMName':
            if row[key] in UTMName_skip:
                pass
            else:
                for v in value:
                    print(v, row[key])
                    if v.lower() in row[key].lower():
                        df.at[index, key] = v
                        break

串流輸出內容已截斷至最後 5000 行。
SalePage 229554
menu_line 229554
SalePage 229554
menu_line 229554
SalePage sales
menu_line sales
SalePage sales
menu_line sales
SalePage 229554
menu_line 229554
SalePage sales
menu_line sales
SalePage sales
menu_line sales
SalePage sales
menu_line sales
SalePage 229554
menu_line 229554
SalePage sales
menu_line sales
SalePage 229554
menu_line 229554
SalePage sales
menu_line sales
SalePage 229554
menu_line 229554
SalePage sales
menu_line sales
SalePage 229554
menu_line 229554
SalePage 229554
menu_line 229554
SalePage 229554
menu_line 229554
SalePage 229554
menu_line 229554
SalePage 229554
menu_line 229554
SalePage 229554
menu_line 229554
SalePage 229554
menu_line 229554
SalePage sale
menu_line sale
SalePage sale
menu_line sale
SalePage sale
menu_line sale
SalePage 229554
menu_line 229554
SalePage sale
menu_line sale
SalePage 229554
menu_line 229554
SalePage 229554
menu_line 229554
SalePage 229554
menu_line 229554
SalePage sale
menu_line sale
SalePage sale
menu_line s

#### check value counts to make sure that the number of categories has decreased, and it was combined successfully

In [20]:
check_value_counts(df)



Behavior
selectcontent        571583
viewproduct          416150
viewmainpage         322815
viewcategory         310954
viewecoupondetail     45445
                      ...  
search                 7175
purchase               5203
viewarticledetail       877
viewcoupondetail        278
register                177
Name: Behavior, Length: 14, dtype: int64

Tunnel
App    1663554
Web     151003
Name: Tunnel, dtype: int64

Device
iOS APP        1109735
Android APP     553819
MobileWeb       100953
Desktop          50050
Name: Device, dtype: int64



Qty
1.0    474260
2.0       312
3.0        40
5.0        30
4.0        20
Name: Qty, dtype: int64


UTMSource


brand.com                534684
Audience                 430103
facebook                 257780
SMS                      128440
System_Inform            121179
TP                        59491
line                      58500
insider                   36495
app_sharing               35290
MemberPresentReward       30146
affiliate                 25832
google                    23895
gkw                       23466
omnichat                  20455
PersonalizedMarketing     10541
brandcom                   7422
mweb                       6018
instagram                  1038
(direct)                    740
AD_CA                       649
gdn                         629
CRM_POP                     463
all                         454
bing                        342
FB                          163
AutoNotification            154
IGS                          65
coupon                       56
instaragm                    22
youtube                      16
vm                           11
s23vip  


HitTime
1677292348639    988
1665718280110    581
1676400687912    574
1677547263030    565
1677513623912    526
                ... 
1666847654139      1
1675861999950      1
1679236229412      1
1672012125716      1
1670369979137      1
Name: HitTime, Length: 187182, dtype: int64


tag
AA    1217821
AP     506479
AL      90257
Name: tag, dtype: int64

viewproduct
0    1398407
1     416150
Name: viewproduct, dtype: int64

selectcontent
0    1242974
1     571583
Name: selectcontent, dtype: int64

viewcategory
0    1503603
1     310954
Name: viewcategory, dtype: int64

viewmainpage
0    1491742
1     322815
Name: viewmainpage, dtype: int64

add
0    1772440
1      42117
Name: add, dtype: int64

viewpromotiondetail
0    1771952
1      42605
Name: viewpromotiondetail, dtype: int64

viewecoupondetail
0    1769112
1      45445
Name: viewecoupondetail, dtype: int64

checkout
0    1802625
1      11932
Name: checkout, dtype: int64

search
0    1807382
1       7175
Name: search, dtype: int64



0112-0129_coupon                                                                          211741
0112-0129_cnygame                                                                         160405
0112-0129_cnysale                                                                          83027
eCoupon_Expiring                                                                           59063
s23-vippreview_m                                                                           53713
1216-1218_viponly                                                                          51088
0823_outletsale                                                                            49724
SalePage                                                                                   40785
TradesOrder                                                                                38333
Timberland                                                                                 25813
MemberTier_2                  

#### [ 資料斷點：可以直接從這裡開始，讀取已合併相似字串的資料.csv ]

In [21]:
df.to_csv('/content/drive/MyDrive/BDA/期末報告/g3/data_preprocessing/midpoint_data/合併相似字串完成_test.csv', index=False)

In [22]:
df = pd.read_csv('/content/drive/MyDrive/BDA/期末報告/g3/data_preprocessing/midpoint_data/合併相似字串完成_test.csv')

#### [ 變數斷點：複製一份處理好的 df，避免後續處理出問題，讓前面必須重跑 ]

In [24]:
df_2 = df.copy()
# remove useless columns
df_2 = df_2.drop('index', axis=1)
df_2

,ShopMemberId,Behavior,Tunnel,Device,PageType,UnitPrice,Qty,TotalSalesAmount,UTMSource,HitTime,EventTime,tag,viewproduct,selectcontent,viewcategory,viewmainpage,add,viewpromotiondetail,viewecoupondetail,checkout,search,viewcustompage,register,purchase,viewcoupondetail,viewarticledetail,UTMSource_Class,UTMMedium,UTMName
0,ec3dkg88Wlr7DGxXWKn4JzZ96BQsIH/JE8RpUte3vqM=,viewcategory,App,iOS APP,0,0.0,NaN,0.0,SMS,1664582733794,1664582734560,AP,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,link,0929_500ecvip
1,ec3dkg88Wlr7DGxXWKn4JzZ96BQsIH/JE8RpUte3vqM=,selectcontent,App,iOS APP,SideBar,0.0,NaN,0.0,SMS,1664582733794,1664582771394,AP,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,link,0929_500ecvip
2,97ZCuM6zfFlGUUJElbAbFGqWggZuMjpqOCtKvR9Fprs=,viewmainpage,App,Android APP,0,0.0,NaN,0.0,facebook,1664582855652,1664582857191,AA,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0926-1011,outletsales
3,97ZCuM6zfFlGUUJElbAbFGqWggZuMjpqOCtKvR9Fprs=,selectcontent,App,Android APP,Home,0.0,NaN,0.0,facebook,1664582855652,1664582859197,AA,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0926-1011,outletsales
4,97ZCuM6zfFlGUUJElbAbFGqWggZuMjpqOCtKvR9Fprs=,selectcontent,App,Android APP,Home,0.0,NaN,0.0,facebook,1664582855652,1664582862966,AA,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0926-1011,outletsales
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1814552,n4b4ediePANQRQf71SRgulrBT9ps9nELe2aZImUDjoY=,selectcontent,App,iOS APP,Home,0.0,NaN,0.0,gkw,1680278268255,1680278386000,AP,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,cpc,ecsite
1814553,tPqhF7GT0P8PBaBOnRtc3smOySs9Cbb3PQmx8FaUbo4=,viewmainpage,App,iOS APP,0,0.0,NaN,0.0,System_Inform,1680278115497,1680278386176,AA,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,E-mail,eCoupon_Expiring
1814554,an9J/uHZwMXusb748tbFuLwG9FAKHAFIFS4xmBRMPs8=,viewproduct,App,iOS APP,0,1800.0,1.0,0.0,Audience,1680277766728,1680278391303,AA,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,AppPush,229554
1814555,n4b4ediePANQRQf71SRgulrBT9ps9nELe2aZImUDjoY=,search,App,iOS APP,0,0.0,NaN,0.0,gkw,1680278268255,1680278391538,AP,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,cpc,ecsite


#### frequency encoding using the new categories

In [25]:
def frequency_encoding(df, freq_enc_columns):
    for col in freq_enc_columns:
        enc_nom = (df.groupby(col).size()) / len(df)
        df[f'{col}_encode'] = df[col].apply(lambda x : enc_nom[x])

In [26]:
freq_enc_columns = ['Behavior', 'Tunnel', 'Device', 'PageType', 'UTMSource', 'UTMMedium', 'UTMName']
frequency_encoding(df_2, freq_enc_columns)
display(df_2)

,ShopMemberId,Behavior,Tunnel,Device,PageType,UnitPrice,Qty,TotalSalesAmount,UTMSource,HitTime,EventTime,tag,viewproduct,selectcontent,viewcategory,viewmainpage,add,viewpromotiondetail,viewecoupondetail,checkout,search,viewcustompage,register,purchase,viewcoupondetail,viewarticledetail,UTMSource_Class,UTMMedium,UTMName,Behavior_encode,Tunnel_encode,Device_encode,PageType_encode,UTMSource_encode,UTMMedium_encode,UTMName_encode
0,ec3dkg88Wlr7DGxXWKn4JzZ96BQsIH/JE8RpUte3vqM=,viewcategory,App,iOS APP,0,0.0,NaN,0.0,SMS,1664582733794,1664582734560,AP,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,link,0929_500ecvip,0.171366,0.916782,0.611574,0.685942,0.070783,0.270581,0.000614
1,ec3dkg88Wlr7DGxXWKn4JzZ96BQsIH/JE8RpUte3vqM=,selectcontent,App,iOS APP,SideBar,0.0,NaN,0.0,SMS,1664582733794,1664582771394,AP,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,link,0929_500ecvip,0.314999,0.916782,0.611574,0.074548,0.070783,0.270581,0.000614
2,97ZCuM6zfFlGUUJElbAbFGqWggZuMjpqOCtKvR9Fprs=,viewmainpage,App,Android APP,0,0.0,NaN,0.0,facebook,1664582855652,1664582857191,AA,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0926-1011,outletsales,0.177903,0.916782,0.305209,0.685942,0.142062,0.003366,0.004899
3,97ZCuM6zfFlGUUJElbAbFGqWggZuMjpqOCtKvR9Fprs=,selectcontent,App,Android APP,Home,0.0,NaN,0.0,facebook,1664582855652,1664582859197,AA,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0926-1011,outletsales,0.314999,0.916782,0.305209,0.183926,0.142062,0.003366,0.004899
4,97ZCuM6zfFlGUUJElbAbFGqWggZuMjpqOCtKvR9Fprs=,selectcontent,App,Android APP,Home,0.0,NaN,0.0,facebook,1664582855652,1664582862966,AA,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0926-1011,outletsales,0.314999,0.916782,0.305209,0.183926,0.142062,0.003366,0.004899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1814552,n4b4ediePANQRQf71SRgulrBT9ps9nELe2aZImUDjoY=,selectcontent,App,iOS APP,Home,0.0,NaN,0.0,gkw,1680278268255,1680278386000,AP,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,cpc,ecsite,0.314999,0.916782,0.611574,0.183926,0.012932,0.045206,0.012932
1814553,tPqhF7GT0P8PBaBOnRtc3smOySs9Cbb3PQmx8FaUbo4=,viewmainpage,App,iOS APP,0,0.0,NaN,0.0,System_Inform,1680278115497,1680278386176,AA,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,E-mail,eCoupon_Expiring,0.177903,0.916782,0.611574,0.685942,0.066782,0.065864,0.032550
1814554,an9J/uHZwMXusb748tbFuLwG9FAKHAFIFS4xmBRMPs8=,viewproduct,App,iOS APP,0,1800.0,1.0,0.0,Audience,1680277766728,1680278391303,AA,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,AppPush,229554,0.229340,0.916782,0.611574,0.685942,0.237029,0.256047,0.002560
1814555,n4b4ediePANQRQf71SRgulrBT9ps9nELe2aZImUDjoY=,search,App,iOS APP,0,0.0,NaN,0.0,gkw,1680278268255,1680278391538,AP,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,cpc,ecsite,0.003954,0.916782,0.611574,0.685942,0.012932,0.045206,0.012932


#### leave only the encoded columns
#### [ 變數斷點：複製一份處理好的 df，避免後續處理出問題，讓前面必須重跑 ]

In [27]:
# leave only the encoded columns
x_filtered_columns = []

for i in target_columns:
    if i in freq_enc_columns:
        x_filtered_columns.append(f'{i}_encode')
    else:
        x_filtered_columns.append(i)
print(x_filtered_columns)

# variable breakpoint
df_3 = df_2[x_filtered_columns]
df_3

['ShopMemberId', 'tag', 'Behavior_encode', 'Tunnel_encode', 'Device_encode', 'PageType_encode', 'UnitPrice', 'TotalSalesAmount', 'UTMSource_encode', 'EventTime', 'UTMMedium_encode', 'UTMName_encode']


,ShopMemberId,tag,Behavior_encode,Tunnel_encode,Device_encode,PageType_encode,UnitPrice,TotalSalesAmount,UTMSource_encode,EventTime,UTMMedium_encode,UTMName_encode
0,ec3dkg88Wlr7DGxXWKn4JzZ96BQsIH/JE8RpUte3vqM=,AP,0.171366,0.916782,0.611574,0.685942,0.0,0.0,0.070783,1664582734560,0.270581,0.000614
1,ec3dkg88Wlr7DGxXWKn4JzZ96BQsIH/JE8RpUte3vqM=,AP,0.314999,0.916782,0.611574,0.074548,0.0,0.0,0.070783,1664582771394,0.270581,0.000614
2,97ZCuM6zfFlGUUJElbAbFGqWggZuMjpqOCtKvR9Fprs=,AA,0.177903,0.916782,0.305209,0.685942,0.0,0.0,0.142062,1664582857191,0.003366,0.004899
3,97ZCuM6zfFlGUUJElbAbFGqWggZuMjpqOCtKvR9Fprs=,AA,0.314999,0.916782,0.305209,0.183926,0.0,0.0,0.142062,1664582859197,0.003366,0.004899
4,97ZCuM6zfFlGUUJElbAbFGqWggZuMjpqOCtKvR9Fprs=,AA,0.314999,0.916782,0.305209,0.183926,0.0,0.0,0.142062,1664582862966,0.003366,0.004899
...,...,...,...,...,...,...,...,...,...,...,...,...
1814552,n4b4ediePANQRQf71SRgulrBT9ps9nELe2aZImUDjoY=,AP,0.314999,0.916782,0.611574,0.183926,0.0,0.0,0.012932,1680278386000,0.045206,0.012932
1814553,tPqhF7GT0P8PBaBOnRtc3smOySs9Cbb3PQmx8FaUbo4=,AA,0.177903,0.916782,0.611574,0.685942,0.0,0.0,0.066782,1680278386176,0.065864,0.032550
1814554,an9J/uHZwMXusb748tbFuLwG9FAKHAFIFS4xmBRMPs8=,AA,0.229340,0.916782,0.611574,0.685942,1800.0,0.0,0.237029,1680278391303,0.256047,0.002560
1814555,n4b4ediePANQRQf71SRgulrBT9ps9nELe2aZImUDjoY=,AP,0.003954,0.916782,0.611574,0.685942,0.0,0.0,0.012932,1680278391538,0.045206,0.012932


#### (2) make y a list

In [28]:
def transform_tag(tag):
    if tag == 'AA':
        return [1, 0, 0]
    elif tag == 'AP':
        return [0, 1, 0]
    elif tag == 'AL':
        return [0, 0, 1]

In [29]:
df_3['y'] = df_3['tag'].apply(lambda x: transform_tag(x))
df_3['y'].value_counts()

[1, 0, 0]    1217821
[0, 1, 0]     506479
[0, 0, 1]      90257
Name: y, dtype: int64

#### (3) normalize columns with big range or standard deviation

In [30]:
# check mean and std
num_col = ['UnitPrice', 'TotalSalesAmount']
for col in num_col:
    print(col)
    print(df_3[col].mean())
    print(df_3[col].std())

UnitPrice
835.2891551491631
1796.1893544794004
TotalSalesAmount
12.718224337951357
301.9869204263717


In [31]:
# normalize columns that have large std
std_list = ['UnitPrice', 'TotalSalesAmount']
for col in std_list:
    df_3[f'{col}_std'] = (df_3[col] - df_3[col].mean()) / df_3[col].std()
display(df_3)

,ShopMemberId,tag,Behavior_encode,Tunnel_encode,Device_encode,PageType_encode,UnitPrice,TotalSalesAmount,UTMSource_encode,EventTime,UTMMedium_encode,UTMName_encode,y,UnitPrice_std,TotalSalesAmount_std
0,ec3dkg88Wlr7DGxXWKn4JzZ96BQsIH/JE8RpUte3vqM=,AP,0.171366,0.916782,0.611574,0.685942,0.0,0.0,0.070783,1664582734560,0.270581,0.000614,"[0, 1, 0]",-0.465034,-0.042115
1,ec3dkg88Wlr7DGxXWKn4JzZ96BQsIH/JE8RpUte3vqM=,AP,0.314999,0.916782,0.611574,0.074548,0.0,0.0,0.070783,1664582771394,0.270581,0.000614,"[0, 1, 0]",-0.465034,-0.042115
2,97ZCuM6zfFlGUUJElbAbFGqWggZuMjpqOCtKvR9Fprs=,AA,0.177903,0.916782,0.305209,0.685942,0.0,0.0,0.142062,1664582857191,0.003366,0.004899,"[1, 0, 0]",-0.465034,-0.042115
3,97ZCuM6zfFlGUUJElbAbFGqWggZuMjpqOCtKvR9Fprs=,AA,0.314999,0.916782,0.305209,0.183926,0.0,0.0,0.142062,1664582859197,0.003366,0.004899,"[1, 0, 0]",-0.465034,-0.042115
4,97ZCuM6zfFlGUUJElbAbFGqWggZuMjpqOCtKvR9Fprs=,AA,0.314999,0.916782,0.305209,0.183926,0.0,0.0,0.142062,1664582862966,0.003366,0.004899,"[1, 0, 0]",-0.465034,-0.042115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1814552,n4b4ediePANQRQf71SRgulrBT9ps9nELe2aZImUDjoY=,AP,0.314999,0.916782,0.611574,0.183926,0.0,0.0,0.012932,1680278386000,0.045206,0.012932,"[0, 1, 0]",-0.465034,-0.042115
1814553,tPqhF7GT0P8PBaBOnRtc3smOySs9Cbb3PQmx8FaUbo4=,AA,0.177903,0.916782,0.611574,0.685942,0.0,0.0,0.066782,1680278386176,0.065864,0.032550,"[1, 0, 0]",-0.465034,-0.042115
1814554,an9J/uHZwMXusb748tbFuLwG9FAKHAFIFS4xmBRMPs8=,AA,0.229340,0.916782,0.611574,0.685942,1800.0,0.0,0.237029,1680278391303,0.256047,0.002560,"[1, 0, 0]",0.537087,-0.042115
1814555,n4b4ediePANQRQf71SRgulrBT9ps9nELe2aZImUDjoY=,AP,0.003954,0.916782,0.611574,0.685942,0.0,0.0,0.012932,1680278391538,0.045206,0.012932,"[0, 1, 0]",-0.465034,-0.042115


### 5. create .npy for RNN
- sort df by by ShopMemberId and EventTime
- create numpy.array to store x values
- create numpy.array to store y values
- create numpy.array to store ShopMemberId_to_index
- save .npy files

#### (1) sort df by by ShopMemberId and EventTime

In [32]:
final_df = df_3.sort_values(by=['ShopMemberId', 'EventTime'], ascending=[True, True])
final_df.drop(['tag', 'UnitPrice', 'TotalSalesAmount'], axis=1, inplace=True)
final_df

,ShopMemberId,Behavior_encode,Tunnel_encode,Device_encode,PageType_encode,UTMSource_encode,EventTime,UTMMedium_encode,UTMName_encode,y,UnitPrice_std,TotalSalesAmount_std
180594,+1MqTgEvnAWYI96utq6PFiX0RihxOmMAeMEnkc+m30k=,0.020526,0.083218,0.055635,0.685942,0.032239,1667450486805,0.008040,0.004517,"[0, 1, 0]",-0.465034,-0.042115
180596,+1MqTgEvnAWYI96utq6PFiX0RihxOmMAeMEnkc+m30k=,0.314999,0.083218,0.055635,0.031867,0.032239,1667450488875,0.008040,0.004517,"[0, 1, 0]",-0.465034,-0.042115
180599,+1MqTgEvnAWYI96utq6PFiX0RihxOmMAeMEnkc+m30k=,0.314999,0.083218,0.055635,0.031867,0.032239,1667450489273,0.008040,0.004517,"[0, 1, 0]",-0.465034,-0.042115
180600,+1MqTgEvnAWYI96utq6PFiX0RihxOmMAeMEnkc+m30k=,0.314999,0.083218,0.055635,0.031867,0.032239,1667450489888,0.008040,0.004517,"[0, 1, 0]",-0.465034,-0.042115
180602,+1MqTgEvnAWYI96utq6PFiX0RihxOmMAeMEnkc+m30k=,0.314999,0.083218,0.055635,0.031867,0.032239,1667450490803,0.008040,0.004517,"[0, 1, 0]",-0.465034,-0.042115
...,...,...,...,...,...,...,...,...,...,...,...,...
1782413,zr/JPzIrZ8jl1IxfFVxJczpS28O8ZtxX4eKQV7QPze8=,0.171366,0.916782,0.611574,0.685942,0.294664,1680116611911,0.270581,0.029601,"[1, 0, 0]",-0.465034,-0.042115
1782414,zr/JPzIrZ8jl1IxfFVxJczpS28O8ZtxX4eKQV7QPze8=,0.177903,0.916782,0.611574,0.685942,0.294664,1680116612781,0.270581,0.029601,"[1, 0, 0]",-0.465034,-0.042115
1782415,zr/JPzIrZ8jl1IxfFVxJczpS28O8ZtxX4eKQV7QPze8=,0.314999,0.916782,0.611574,0.183926,0.294664,1680116614640,0.270581,0.029601,"[1, 0, 0]",-0.465034,-0.042115
1782416,zr/JPzIrZ8jl1IxfFVxJczpS28O8ZtxX4eKQV7QPze8=,0.314999,0.916782,0.611574,0.074548,0.294664,1680116621118,0.270581,0.029601,"[1, 0, 0]",-0.465034,-0.042115


#### (2) create numpy.array to store x values by event and ShopMemberId

In [33]:
x_col_list = ['EventTime', 'Behavior_encode', 'Tunnel_encode', 'Device_encode', 'PageType_encode', 'UnitPrice_std', 'TotalSalesAmount_std',	'UTMSource_encode',	'UTMMedium_encode', 'UTMName_encode']
final_df['x_values'] = final_df.apply(lambda x: np.array([x[c] for c in x_col_list]), axis=1)
final_df['ShopMemberId_to_index'] = np.array(final_df['ShopMemberId'].astype('category').cat.codes, dtype=np.float)
final_df

,ShopMemberId,Behavior_encode,Tunnel_encode,Device_encode,PageType_encode,UTMSource_encode,EventTime,UTMMedium_encode,UTMName_encode,y,UnitPrice_std,TotalSalesAmount_std,x_values,ShopMemberId_to_index
180594,+1MqTgEvnAWYI96utq6PFiX0RihxOmMAeMEnkc+m30k=,0.020526,0.083218,0.055635,0.685942,0.032239,1667450486805,0.008040,0.004517,"[0, 1, 0]",-0.465034,-0.042115,"[1667450486805.0, 0.020526222102695037, 0.0832...",0.0
180596,+1MqTgEvnAWYI96utq6PFiX0RihxOmMAeMEnkc+m30k=,0.314999,0.083218,0.055635,0.031867,0.032239,1667450488875,0.008040,0.004517,"[0, 1, 0]",-0.465034,-0.042115,"[1667450488875.0, 0.3149986470526966, 0.083217...",0.0
180599,+1MqTgEvnAWYI96utq6PFiX0RihxOmMAeMEnkc+m30k=,0.314999,0.083218,0.055635,0.031867,0.032239,1667450489273,0.008040,0.004517,"[0, 1, 0]",-0.465034,-0.042115,"[1667450489273.0, 0.3149986470526966, 0.083217...",0.0
180600,+1MqTgEvnAWYI96utq6PFiX0RihxOmMAeMEnkc+m30k=,0.314999,0.083218,0.055635,0.031867,0.032239,1667450489888,0.008040,0.004517,"[0, 1, 0]",-0.465034,-0.042115,"[1667450489888.0, 0.3149986470526966, 0.083217...",0.0
180602,+1MqTgEvnAWYI96utq6PFiX0RihxOmMAeMEnkc+m30k=,0.314999,0.083218,0.055635,0.031867,0.032239,1667450490803,0.008040,0.004517,"[0, 1, 0]",-0.465034,-0.042115,"[1667450490803.0, 0.3149986470526966, 0.083217...",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1782413,zr/JPzIrZ8jl1IxfFVxJczpS28O8ZtxX4eKQV7QPze8=,0.171366,0.916782,0.611574,0.685942,0.294664,1680116611911,0.270581,0.029601,"[1, 0, 0]",-0.465034,-0.042115,"[1680116611911.0, 0.17136634451273783, 0.91678...",8230.0
1782414,zr/JPzIrZ8jl1IxfFVxJczpS28O8ZtxX4eKQV7QPze8=,0.177903,0.916782,0.611574,0.685942,0.294664,1680116612781,0.270581,0.029601,"[1, 0, 0]",-0.465034,-0.042115,"[1680116612781.0, 0.17790292616875633, 0.91678...",8230.0
1782415,zr/JPzIrZ8jl1IxfFVxJczpS28O8ZtxX4eKQV7QPze8=,0.314999,0.916782,0.611574,0.183926,0.294664,1680116614640,0.270581,0.029601,"[1, 0, 0]",-0.465034,-0.042115,"[1680116614640.0, 0.3149986470526966, 0.916782...",8230.0
1782416,zr/JPzIrZ8jl1IxfFVxJczpS28O8ZtxX4eKQV7QPze8=,0.314999,0.916782,0.611574,0.074548,0.294664,1680116621118,0.270581,0.029601,"[1, 0, 0]",-0.465034,-0.042115,"[1680116621118.0, 0.3149986470526966, 0.916782...",8230.0


In [34]:
x_testing = final_df['x_values'].values
x_testing

array([array([ 1.66745049e+12,  2.05262221e-02,  8.32175567e-02,  5.56350669e-02,
               6.85942078e-01, -4.65034019e-01, -4.21151496e-02,  3.22392738e-02,
               8.03997890e-03,  4.51735603e-03])                                 ,
       array([ 1.66745049e+12,  3.14998647e-01,  8.32175567e-02,  5.56350669e-02,
               3.18667311e-02, -4.65034019e-01, -4.21151496e-02,  3.22392738e-02,
               8.03997890e-03,  4.51735603e-03])                                 ,
       array([ 1.66745049e+12,  3.14998647e-01,  8.32175567e-02,  5.56350669e-02,
               3.18667311e-02, -4.65034019e-01, -4.21151496e-02,  3.22392738e-02,
               8.03997890e-03,  4.51735603e-03])                                 ,
       ...,
       array([ 1.68011661e+12,  3.14998647e-01,  9.16782443e-01,  6.11573514e-01,
               1.83926435e-01, -4.65034019e-01, -4.21151496e-02,  2.94663656e-01,
               2.70580643e-01,  2.96011644e-02])                                 ,


In [35]:
float_x_arr = np.vstack(x_testing[:]).astype(np.float)
float_x_arr

array([[1.66745049e+12, 2.05262221e-02, 8.32175567e-02, ...,
        3.22392738e-02, 8.03997890e-03, 4.51735603e-03],
       [1.66745049e+12, 3.14998647e-01, 8.32175567e-02, ...,
        3.22392738e-02, 8.03997890e-03, 4.51735603e-03],
       [1.66745049e+12, 3.14998647e-01, 8.32175567e-02, ...,
        3.22392738e-02, 8.03997890e-03, 4.51735603e-03],
       ...,
       [1.68011661e+12, 3.14998647e-01, 9.16782443e-01, ...,
        2.94663656e-01, 2.70580643e-01, 2.96011644e-02],
       [1.68011662e+12, 3.14998647e-01, 9.16782443e-01, ...,
        2.94663656e-01, 2.70580643e-01, 2.96011644e-02],
       [1.68011662e+12, 1.71366345e-01, 9.16782443e-01, ...,
        2.94663656e-01, 2.70580643e-01, 2.96011644e-02]])

In [36]:
float_x_arr.shape

(1814557, 10)

#### (3) create numpy.array to store y values by ShopMemberId

In [37]:
y_training_new = np.array(final_df['y'].apply(lambda x: np.array(x, dtype=float)))
y_training_new

array([array([0., 1., 0.]), array([0., 1., 0.]), array([0., 1., 0.]), ...,
       array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.])],
      dtype=object)

In [38]:
float_y_arr = np.vstack(y_training_new[:]).astype(np.float)
float_y_arr

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [39]:
float_y_arr.shape

(1814557, 3)

#### (4) create numpy array to store ShopMemberId_to_index

In [40]:
testing_ids = final_df['ShopMemberId_to_index'].values.astype(np.float)
testing_ids

array([   0.,    0.,    0., ..., 8230., 8230., 8230.])

In [41]:
testing_ids.dtype

dtype('float64')

#### (5) save .npy files

In [42]:
np.save('/content/drive/MyDrive/BDA/期末報告/g3/data_preprocessing/final_output/x_testing.npy', float_x_arr)

In [43]:
np.save('/content/drive/MyDrive/BDA/期末報告/g3/data_preprocessing/final_output/y_testing.npy', float_y_arr)

In [44]:
np.save('/content/drive/MyDrive/BDA/期末報告/g3/data_preprocessing/final_output/testing_ids.npy', testing_ids)